In [ ]:
%load_ext autoreload
%autoreload 2
import kfp.dsl as dsl
import os
from dotenv import load_dotenv
from kfp.v2.dsl import Output, component, InputPath, HTML, Input, Dataset
from kfp.v2 import compiler
from google.cloud import aiplatform as aip
load_dotenv()

bucket = os.getenv('bucket')
gcp_project = os.getenv('gcp_project')
gcp_service_account = os.getenv('gcp_service_account')

### Check that we have access 

In [ ]:
aip.init(
    project=gcp_project,
    location="europe-west1",
)
all_piplines = aip.PipelineJob.list()
all_piplines

In [ ]:
def train_model(input:int)->float: 
    return 2+input

def ingetst_data()->float: 
    return 2

ingest_data_component = component(
    ingetst_data
)

train_component = component(
    train_model
)

@dsl.pipeline(
    name="ltv-train",
)
def add_pipeline(
):

  first_add_task = ingest_data_component()

  second_add_task = train_component(2)

compiler.Compiler().compile(pipeline_func=add_pipeline, package_path="local_run.json")

In [ ]:
job = aip.PipelineJob(
    display_name="test",
    template_path="local_run.json",
    pipeline_root=bucket,
    location="europe-west1",
    project=gcp_project,
)

job.submit(service_account=gcp_service_account)